In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import pickle
import re
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("crudg.json", scope)
client = gspread.authorize(creds)
sheet = client.open("flightsched").get_worksheet(2)
dataframe = pd.DataFrame(sheet.get_all_records())


In [20]:
# Set ipython's max row display
pd.set_option('display.max_row', 50)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [21]:

dataframe["Modeffdate"]= pd.to_datetime(dataframe["Modeffdate"], unit="s")
dataframe["moddisdate"]= pd.to_datetime(dataframe["moddisdate"], unit="s")
dataframe["moddeptime"]= pd.to_datetime(dataframe["moddeptime"], unit="s")
dataframe["mod_arr_time"]= pd.to_datetime(dataframe["mod_arr_time"], unit="s")
dataframe = dataframe.drop(columns=["Dept Time" ,"Arrv Time","Subfleet","Service Type","Eff Date","Dis Date","01-Jan-1970"])
dataframe["Freq"] = dataframe["Freq"].astype(str)
dataframe["Freq"] = dataframe["Freq"].str.replace(".","")
dataframe["engg_arr"]= False
dataframe["engg_dep"]=False
dataframe["deptype"]=False
dataframe["actual_days"]=""
cols = [ 'Dept Arp', 'engg_arr','engg_dep', 'deptype','Arvl Arp', 'Flt Desg',   'moddeptime', 'mod_arr_time'  ,'Text Comment', 'Freq','Block Time','Modeffdate', 'moddisdate',"actual_days"
,]
dataframe = dataframe[cols]

#  modyfying actualdays column for complete dataset 

for xray in range(0,len(dataframe)):
    bravo =  pd.to_datetime(str(dataframe.loc[xray,"Modeffdate"]))
    while dataframe.loc[xray,"moddisdate"] >= bravo:
        # one added as day of the week is different in pandas and data 
        if str(bravo.dayofweek + 1) in dataframe.loc[xray,"Freq"]:
            dataframe.loc[xray,"actual_days"]+= "/"+ str(bravo.day)
        bravo+= pd.DateOffset(1)

# adding columns for each day of month 
col_name_list=[]
nilList=[]
StrArr = "A"
StrDep ="D"
for india in range(1,32):
     col_name_list.append(StrDep + str(india))
     col_name_list.append(StrArr + str(india))
     nilList.append(False)
     nilList.append(False)
cols = cols +col_name_list
dataframe = pd.concat([dataframe,pd.DataFrame([nilList],index=dataframe.index,columns = col_name_list)], axis=1)



In [22]:
onedf=dataframe.loc[(dataframe["Text Comment"]=="A83") & (dataframe["Dept Arp"]=="STV")]
# onedf =onedf.sort_values(by="value")
onedf.head(30)

,Dept Arp,engg_arr,engg_dep,deptype,Arvl Arp,Flt Desg,moddeptime,mod_arr_time,Text Comment,Freq,Block Time,Modeffdate,moddisdate,actual_days,D1,A1,D2,A2,D3,A3,D4,A4,D5,A5,D6,...,A19,D20,A20,D21,A21,D22,A22,D23,A23,D24,A24,D25,A25,D26,A26,D27,A27,D28,A28,D29,A29,D30,A30,D31,A31
1297,STV,False,False,False,HYD,6E 0497,1970-01-01 12:00:00,1970-01-01 13:30:00,A83,1234567,1:30,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [23]:
test_df = dataframe.head(30)

In [24]:

# first day of month set manually need to to check edge cases 
def remove_duplicate_flights (x_df):
      temp_dict ={}
      
      for xray in range(0,len(x_df)):
            if not (str(x_df.loc[xray,'Flt Desg']) in temp_dict):
                  temp_dict[x_df.loc[xray,'Flt Desg']] =xray
                  alpha = pd.to_datetime(str(x_df.loc[xray,"Modeffdate"].date()) +" "+ x_df.loc[xray,"moddeptime"].strftime('%H:%M:%S'),format="%Y-%m-%d %H:%M:%S").replace(day=1)
                  beta = pd.to_datetime(str(x_df.loc[xray,"Modeffdate"].date()) +" "+ x_df.loc[xray,"mod_arr_time"].strftime('%H:%M:%S'),format="%Y-%m-%d %H:%M:%S").replace(day=1)
                  for india in range(1,32):
                        if str(india) in x_df.loc[xray,'actual_days'].split("/"):
                              x_df.loc[xray,"D"+str(india)] = alpha
                              x_df.loc[xray,"A"+str(india)] = beta
                        alpha+=pd.DateOffset(1)
                        beta+=pd.DateOffset(1)

            elif (str(x_df.loc[xray,'Flt Desg']) in temp_dict):
                  alpha = pd.to_datetime(str(x_df.loc[xray,"Modeffdate"].date()) +" "+ x_df.loc[xray,"moddeptime"].strftime('%H:%M:%S'),format="%Y-%m-%d %H:%M:%S").replace(day=1)
                  beta = pd.to_datetime(str(x_df.loc[xray,"Modeffdate"].date()) +" "+ x_df.loc[xray,"mod_arr_time"].strftime('%H:%M:%S'),format="%Y-%m-%d %H:%M:%S").replace(day=1)              
                  zulu = temp_dict[x_df.loc[xray,'Flt Desg']]
                  
                  for india in range(1,32):
                        if str(india) in x_df.loc[xray,'actual_days'].split("/"):
                              x_df.loc[zulu,"D"+str(india)] = alpha
                              x_df.loc[zulu,"A"+str(india)] = beta
                        alpha+=pd.DateOffset(1)
                        beta+=pd.DateOffset(1)

      
      
                 







In [25]:
# temporray cell 
onedf=dataframe.loc[(dataframe["Text Comment"] == "A83" ) | (dataframe["Text Comment"] == "A2" ) ]
# onedf =onedf.sort_values(by="value")
onedf.head(30)

,Dept Arp,engg_arr,engg_dep,deptype,Arvl Arp,Flt Desg,moddeptime,mod_arr_time,Text Comment,Freq,Block Time,Modeffdate,moddisdate,actual_days,D1,A1,D2,A2,D3,A3,D4,A4,D5,A5,D6,...,A19,D20,A20,D21,A21,D22,A22,D23,A23,D24,A24,D25,A25,D26,A26,D27,A27,D28,A28,D29,A29,D30,A30,D31,A31
415,MAA,False,False,False,HYD,6E 0384,1970-01-01 06:40:00,1970-01-01 07:50:00,A2,1234567,1:10,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
416,HYD,False,False,False,BOM,6E 0384,1970-01-01 08:55:00,1970-01-01 10:20:00,A2,1234567,1:25,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
417,BOM,False,False,False,HYD,6E 6117,1970-01-01 12:00:00,1970-01-01 13:35:00,A2,1234567,1:35,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
418,HYD,False,False,False,MAA,6E 0497,1970-01-01 14:45:00,1970-01-01 16:10:00,A2,1234567,1:25,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
419,MAA,False,False,False,DEL,6E 2752,1970-01-01 17:05:00,1970-01-01 20:00:00,A2,1234567,2:55,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
420,DEL,False,False,False,MAA,6E 2519,1970-01-01 20:45:00,1970-01-01 23:30:00,A2,1234567,2:45,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1294,HYD,False,False,False,PNQ,6E 6516,1970-01-01 09:00:00,1970-01-01 10:00:00,A83,1357,1:00,2021-02-01,2021-02-28,/1/3/5/7/8/10/12/14/15/17/19/21/22/24/26/28,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1295,HYD,False,False,False,STV,6E 0493,1970-01-01 09:35:00,1970-01-01 11:20:00,A83,1234567,1:45,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1296,PNQ,False,False,False,HYD,6E 6517,1970-01-01 10:55:00,1970-01-01 12:05:00,A83,1357,1:10,2021-02-01,2021-02-28,/1/3/5/7/8/10/12/14/15/17/19/21/22/24/26/28,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1297,STV,False,False,False,HYD,6E 0497,1970-01-01 12:00:00,1970-01-01 13:30:00,A83,1234567,1:30,2021-02-01,2021-02-28,/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/...,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,Fals

In [8]:






remove_duplicate_flights(dataframe)
dataframe = dataframe.drop_duplicates(subset=['Flt Desg','Text Comment'],keep='first')

# removing columns
dataframe = dataframe.drop(['moddeptime', 'mod_arr_time', 'Freq',
                    'Modeffdate', 'moddisdate', 'actual_days','engg_dep'], axis=1)

# pivoting the dataset 
dataframe = pd.melt(dataframe, id_vars=['Dept Arp', 'engg_arr','deptype', 'Arvl Arp', 'Flt Desg',
                                'Text Comment', 'Block Time'], value_vars=['D1', 'A1', 'D2', 'A2', 'D3', 'A3', 'D4',
                                                                           'A4', 'D5', 'A5', 'D6', 'A6', 'D7', 'A7', 'D8', 'A8', 'D9', 'A9', 'D10',
                                                                           'A10', 'D11', 'A11', 'D12', 'A12', 'D13', 'A13', 'D14', 'A14', 'D15',
                                                                           'A15', 'D16', 'A16', 'D17', 'A17', 'D18', 'A18', 'D19', 'A19', 'D20',
                                                                           'A20', 'D21', 'A21', 'D22', 'A22', 'D23', 'A23', 'D24', 'A24', 'D25',
                                                                           'A25', 'D26', 'A26', 'D27', 'A27', 'D28', 'A28', 'D29', 'A29', 'D30',
                                                                           'A30', 'D31', 'A31'])


In [9]:
# Dropping dataframe with value equals false 

dataframe = dataframe[dataframe.value != False]

In [10]:
# List of airports alphabetically 
dep_airport_list = list(dataframe['Dept Arp'].unique())
dep_airport_list.sort()
dep_airport_list.append("All_Stations")
# List of days daywise
days_sorted_list= list(filter(lambda x: "A" not in x ,col_name_list))
days_sorted_list
dep_airport_list
dataframe = dataframe.reset_index(drop=True)
pickle_List = [dataframe,days_sorted_list,dep_airport_list]

In [11]:
# Pickle the dataset
pickle.dump(pickle_List, open("gsheet_pickle.p", "wb"))

alpha = pickle.load(open("gsheet_pickle.p", "rb"))

alpha[0].equals(dataframe)

True

In [12]:

# dataframe from pickle list 
req_df = alpha[0]
req_df.describe()
req_df.head(15)


,Dept Arp,engg_arr,deptype,Arvl Arp,Flt Desg,Text Comment,Block Time,variable,value
0,IXC,False,False,GOI,6E 0389,A10,2:45,D1,2021-02-01 14:25:00
1,GOI,False,False,IXC,6E 0391,A10,2:40,D1,2021-02-01 17:40:00
2,BLR,False,False,RPR,6E 0378,A100,2:00,D1,2021-02-01 08:30:00
3,RPR,False,False,BLR,6E 0382,A100,1:35,D1,2021-02-01 11:10:00
4,BLR,False,False,VGA,6E 0507,A100,1:20,D1,2021-02-01 13:20:00
5,VGA,False,False,BLR,6E 0506,A100,1:20,D1,2021-02-01 16:00:00
6,BLR,False,False,GOI,6E 0636,A100,1:20,D1,2021-02-01 18:20:00
7,GOI,False,False,BLR,6E 0369,A100,1:20,D1,2021-02-01 20:20:00
8,CCU,False,False,DIB,6E 0455,A101,1:35,D1,2021-02-01 11:20:00
9,DIB,False,False,CCU,6E 0206,A101,2:00,D1,2021-02-01 13:30:00


In [13]:
# manual testing 
test_alpha = req_df.sample(n=5)
test_alpha

,Dept Arp,engg_arr,deptype,Arvl Arp,Flt Desg,Text Comment,Block Time,variable,value
2525,UDR,False,False,BOM,6E 5324,A320_789,1:45,D2,2021-02-02 13:40:00
37519,DEL,False,False,BLR,6E 2297,A75,2:45,D18,2021-02-18 13:35:00
30842,VTZ,False,False,BLR,6E 0524,A56,1:50,D15,2021-02-15 18:55:00
24525,TRZ,False,False,MAA,6E 7151,ATR008,1:10,D12,2021-02-12 22:15:00
53307,DEL,False,False,CJB,6E 6908,A130,2:55,A25,2021-02-25 21:35:00


In [14]:
# manual testing continued as cell excution creates a new random sample 
onedf=req_df.loc[(req_df["Text Comment"]=="A83") & (req_df["Dept Arp"]=="STV")]
onedf =onedf.sort_values(by="value")
onedf

,Dept Arp,engg_arr,deptype,Arvl Arp,Flt Desg,Text Comment,Block Time,variable,value


In [15]:

onedf=dataframe.loc[dataframe["Text Comment"]=="A10"]

# sorting and operting on one rotation 

onedf =onedf.sort_values("moddeptime")


onedf["deptype"].iloc[0] ="baseDep"
onedf["deptype"].iloc[-1] ="baseArr"
mylist = onedf['Flt Desg'].values.tolist()
indices_base_dep = [i for i, x in enumerate(mylist) if x == mylist[0]]
indices_base_arr = [i for i, x in enumerate(mylist) if x == mylist[-1]]
for india in indices_base_dep:
    onedf["deptype"].iloc[india] ="baseDep"
for xray in indices_base_arr:
    onedf["deptype"].iloc[xray] ="baseArr"


# adding  a duplicate row at the bottom for base arival Aircraft 
is_Base_Arrival = onedf["deptype"] =="baseArr"
onedf_try=onedf[is_Base_Arrival]
onedf=onedf.append(onedf_try,ignore_index=True)


# modifying the eng_arr and eng_dep for one rotation 
for xray in range(0,len(onedf)):
    if onedf.loc[xray,"deptype"] == "baseDep":
        onedf.loc[xray,"engg_arr"] = "baseAcft"
        onedf.loc[xray,"engg_dep"] = onedf.loc[xray,"moddeptime"]
    elif onedf.loc[xray,"deptype"] != "baseDep" and onedf.loc[xray,"deptype"] != "baseArr":
        onedf.loc[xray,"engg_arr"] = onedf.loc[xray-1,"mod_arr_time"] 
        onedf.loc[xray,"engg_dep"] = onedf.loc[xray,"moddeptime"] 



KeyError: 'moddeptime'